In [10]:
from transformers import AutoTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torchaudio
from datasets import load_dataset
import re
import jiwer

In [3]:
# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian")
model = Wav2Vec2ForCTC.from_pretrained("Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian")

/home/redalexdad/anaconda3/envs/dl_science/lib/python3.9/site-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a

In [7]:
# Загрузка аудиофайла
audio_path = "./voice/000000_RUSLAN.wav"
resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)
speech, _ = torchaudio.load(audio_path)
input_values = resampler.forward(speech.squeeze(0)).numpy()

In [11]:
# Токенизация и получение предсказания от модели
input_values = torch.tensor(input_values).unsqueeze(0)  # добавляем размерность пакета
with torch.no_grad():
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

In [12]:
# Декодирование предсказанных токенов в текст
transcription = tokenizer.batch_decode(predicted_ids)[0]

In [13]:
# Ваш код для загрузки и обработки субтитров
# Например, вы можете использовать библиотеку jiwer для вычисления Word Error Rate (WER)
# Убедитесь, что у вас есть правильные субтитры для сравнения
reference_text = "ваш текст субтитров"
wer_score = jiwer.wer(reference_text, transcription)
print("Word Error Rate (WER): {:.2f}%".format(wer_score*100))

Word Error Rate (WER): 133.33%
